In [31]:
!pip install --upgrade pip
!pip install pmdarima
!pip install psycopg2-binary

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at 

In [50]:
#Import Python Packages
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import math
from datetime import datetime, timedelta
import pandas as pd
import time
import psycopg2
import pmdarima as pm
import boto3

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

In [51]:
# Put the ARN of your AWS Secrets Manager secret for your redshift cluster here:
secret_name="arn:aws:secretsmanager:us-west-2:451614664738:secret:ForecastingSecret-JRCdLL"

# This will get the secret from AWS Secrets Manager. 
import boto3
import json

session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager'
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)

if 'SecretString' in get_secret_value_response:
    connection_info = json.loads(get_secret_value_response['SecretString'])
else:
    print("ERROR: no secret data found")

In [52]:

credentials=psycopg2.connect(
    dbname   = 'exelredshift', 
    host     = connection_info["host"], 
    port     = connection_info["port"], 
    user     = connection_info["username"], 
    password = connection_info["password"]
)

In [53]:
# Read in your SQL query results using pandas

df= pd.read_sql("""
            select * from edm.bl_cabo_rcc_hcc_nps
            """, con = credentials)



In [54]:
selection_rcc_comm = df[['week_ending_dt','projected_sp_rcc_sales','projected_sd_rcc_sales']]
selection_hcc_comm = df[['week_ending_dt','projected_sp_hcc_sales','projected_sd_hcc_sales']]
selection_dtc_comm = df[['week_ending_dt','projected_sp_thy_sales','projected_sd_thy_sales']]
selection_ease = df[['week_ending_dt','projected_ease_rcc_sales','projected_ease_hcc_sales','projected_ease_thy_sales']]


In [55]:

sorted_data_rcc_comm= selection_rcc_comm.sort_values(by='week_ending_dt')
sorted_data_hcc_comm= selection_hcc_comm.sort_values(by='week_ending_dt')
sorted_data_dtc_comm= selection_dtc_comm.sort_values(by='week_ending_dt')
sorted_data_ease = selection_ease.sort_values(by='week_ending_dt')



In [56]:

rcc_agg_df= sorted_data_rcc_comm.groupby(["week_ending_dt"]).sum()
hcc_agg_df= sorted_data_hcc_comm.groupby(["week_ending_dt"]).sum()
dtc_agg_df= sorted_data_dtc_comm.groupby(["week_ending_dt"]).sum()
ease_agg_df= sorted_data_ease.groupby(["week_ending_dt"]).sum()



In [10]:
rcc_agg_sorted = rcc_agg_df.sort_values(by='week_ending_dt')
hcc_agg_sorted = hcc_agg_df.sort_values(by='week_ending_dt')
dtc_agg_sorted = dtc_agg_df.sort_values(by='week_ending_dt')
ease_agg_sorted = ease_agg_df.sort_values(by='week_ending_dt')

In [11]:
rcc_dfn= rcc_agg_sorted.reset_index()
hcc_dfn= hcc_agg_sorted.reset_index()
dtc_dfn= dtc_agg_sorted.reset_index()
ease_dfn= ease_agg_sorted.reset_index()

In [12]:

rcc_dfn['RCC_Outs'] = rcc_dfn['projected_sp_rcc_sales']+ rcc_dfn['projected_sd_rcc_sales']
hcc_dfn['HCC_Outs'] = hcc_dfn['projected_sp_hcc_sales']+ hcc_dfn['projected_sd_hcc_sales']
dtc_dfn['DTC_Outs'] = dtc_dfn['projected_sp_thy_sales']+ dtc_dfn['projected_sd_thy_sales']
ease_dfn['EASE_Outs'] = ease_dfn['projected_ease_rcc_sales']+ ease_dfn['projected_ease_hcc_sales']+ ease_dfn['projected_ease_thy_sales']


In [13]:

rcc_new_dfn= rcc_dfn[['week_ending_dt', 'RCC_Outs']]
hcc_new_dfn= hcc_dfn[['week_ending_dt', 'HCC_Outs']]
dtc_new_dfn= dtc_dfn[['week_ending_dt', 'DTC_Outs']]
ease_new_dfn= ease_dfn[['week_ending_dt', 'EASE_Outs']]


In [14]:

#Convert to datetime format
rcc_new_dfn['week_end_date'] = pd.to_datetime(rcc_new_dfn['week_ending_dt'])
hcc_new_dfn['week_end_date'] = pd.to_datetime(hcc_new_dfn['week_ending_dt'])
dtc_new_dfn['week_end_date'] = pd.to_datetime(dtc_new_dfn['week_ending_dt'])
ease_new_dfn['week_end_date'] = pd.to_datetime(ease_new_dfn['week_ending_dt'])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [15]:
rcc_sorted_dfn= rcc_new_dfn.sort_values(by='week_end_date')
rcc_sorted_dfn.reset_index()

hcc_sorted_dfn= hcc_new_dfn.sort_values(by='week_end_date')
hcc_sorted_dfn.reset_index()

dtc_sorted_dfn= dtc_new_dfn.sort_values(by='week_end_date')
dtc_sorted_dfn.reset_index()

ease_sorted_dfn= ease_new_dfn.sort_values(by='week_end_date')
ease_sorted_dfn.reset_index()

,index,week_ending_dt,EASE_Outs,week_end_date
0,0,2018-01-05,15.50,2018-01-05
1,1,2018-01-12,41.50,2018-01-12
2,2,2018-01-19,37.97,2018-01-19
3,3,2018-01-26,38.96,2018-01-26
4,4,2018-02-02,69.50,2018-02-02
...,...,...,...,...
285,285,2023-06-23,155.50,2023-06-23
286,286,2023-06-30,195.50,2023-06-30
287,287,2023-07-07,149.00,2023-07-07
288,288,2023-07-14,160.50,2023-07-14


In [16]:
rcc_master_dataframe_comm= rcc_sorted_dfn[['week_end_date','RCC_Outs']]
hcc_master_dataframe_comm= hcc_sorted_dfn[['week_end_date','HCC_Outs']]
dtc_master_dataframe_comm= dtc_sorted_dfn[['week_end_date','DTC_Outs']]
ease_master_dataframe_comm= ease_sorted_dfn[['week_end_date','EASE_Outs']]

In [17]:
rcc_master_dataframe_comm['RCC_Outs'] = rcc_master_dataframe_comm['RCC_Outs'].apply(np.ceil)
hcc_master_dataframe_comm['HCC_Outs'] = hcc_master_dataframe_comm['HCC_Outs'].apply(np.ceil)
dtc_master_dataframe_comm['DTC_Outs'] = dtc_master_dataframe_comm['DTC_Outs'].apply(np.ceil)
ease_master_dataframe_comm['EASE_Outs'] = ease_master_dataframe_comm['EASE_Outs'].apply(np.ceil)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [18]:
rcc_master_dataframe_comm = rcc_master_dataframe_comm.astype({"RCC_Outs": int})
hcc_master_dataframe_comm = hcc_master_dataframe_comm.astype({"HCC_Outs": int})
dtc_master_dataframe_comm = dtc_master_dataframe_comm.astype({"DTC_Outs": int})
ease_master_dataframe_comm = ease_master_dataframe_comm.astype({"EASE_Outs": int})

In [19]:
rcc_master_dataframe_cabo = rcc_master_dataframe_comm[['week_end_date','RCC_Outs']]
hcc_master_dataframe_cabo = hcc_master_dataframe_comm[['week_end_date','HCC_Outs']]
dtc_master_dataframe_cabo = dtc_master_dataframe_comm[['week_end_date','DTC_Outs']]
ease_master_dataframe_cabo = ease_master_dataframe_comm[['week_end_date','EASE_Outs']]

In [20]:
rcc_master_dataframe_cabo['week_end_date']=pd.to_datetime(rcc_master_dataframe_cabo['week_end_date'])
hcc_master_dataframe_cabo['week_end_date']=pd.to_datetime(hcc_master_dataframe_cabo['week_end_date'])
dtc_master_dataframe_cabo['week_end_date']=pd.to_datetime(dtc_master_dataframe_cabo['week_end_date'])
ease_master_dataframe_cabo['week_end_date']=pd.to_datetime(ease_master_dataframe_cabo['week_end_date'])

In [21]:
#Set date column as the index
rcc_new_df1 = rcc_master_dataframe_cabo[rcc_master_dataframe_cabo['week_end_date'] > '2021-01-01']
hcc_new_df1 = hcc_master_dataframe_cabo[hcc_master_dataframe_cabo['week_end_date'] > '2021-01-01']
dtc_new_df1 = dtc_master_dataframe_cabo[dtc_master_dataframe_cabo['week_end_date'] > '2021-01-01']
ease_new_df1 = ease_master_dataframe_cabo[ease_master_dataframe_cabo['week_end_date'] > '2021-01-01']

In [22]:
#Set date column as the index
rcc_new_df =rcc_new_df1.set_index('week_end_date')
hcc_new_df =hcc_new_df1.set_index('week_end_date')
dtc_new_df =dtc_new_df1.set_index('week_end_date')
ease_new_df =ease_new_df1.set_index('week_end_date')

In [23]:
rcc_a = rcc_master_dataframe_cabo.tail(1)
hcc_a = hcc_master_dataframe_cabo.tail(1)
dtc_a = dtc_master_dataframe_cabo.tail(1)
ease_a = ease_master_dataframe_cabo.tail(1)

In [24]:
#Training Data Selection

rcc_train  = rcc_new_df[(rcc_new_df.index.get_level_values(0) >= rcc_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (rcc_new_df.index.get_level_values(0) < rcc_a['week_end_date'].iloc[0])]

hcc_train  = hcc_new_df[(hcc_new_df.index.get_level_values(0) >= hcc_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (hcc_new_df.index.get_level_values(0) < hcc_a['week_end_date'].iloc[0])]

dtc_train  = dtc_new_df[(dtc_new_df.index.get_level_values(0) >= dtc_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (dtc_new_df.index.get_level_values(0) < dtc_a['week_end_date'].iloc[0])]

ease_train = ease_new_df[(ease_new_df.index.get_level_values(0) >= ease_a['week_end_date'].iloc[0] - timedelta(days=736) ) & (ease_new_df.index.get_level_values(0) < ease_a['week_end_date'].iloc[0])]



In [25]:


#RCC model
rcc_model = pm.auto_arima(rcc_new_df.RCC_Outs.iloc[:-1], 
                        m=52, seasonal=True,
                      start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True)



Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[52] intercept   : AIC=inf, Time=2.57 sec
 ARIMA(0,1,0)(0,0,0)[52] intercept   : AIC=1730.130, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[52] intercept   : AIC=1643.876, Time=1.05 sec
 ARIMA(0,1,1)(0,0,1)[52] intercept   : AIC=1613.477, Time=1.18 sec
 ARIMA(0,1,0)(0,0,0)[52]             : AIC=1728.216, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[52] intercept   : AIC=1619.300, Time=0.08 sec
 ARIMA(0,1,1)(1,0,1)[52] intercept   : AIC=inf, Time=2.06 sec
 ARIMA(0,1,1)(0,0,2)[52] intercept   : AIC=inf, Time=7.64 sec
 ARIMA(0,1,1)(1,0,0)[52] intercept   : AIC=1609.158, Time=1.12 sec
 ARIMA(0,1,1)(2,0,0)[52] intercept   : AIC=1603.186, Time=7.94 sec
 ARIMA(0,1,1)(2,0,1)[52] intercept   : AIC=inf, Time=12.18 sec
 ARIMA(0,1,0)(2,0,0)[52] intercept   : AIC=inf, Time=4.98 sec
 ARIMA(1,1,1)(2,0,0)[52] intercept   : AIC=1592.250, Time=9.11 sec
 ARIMA(1,1,1)(1,0,0)[52] intercept   : AIC=1597.303, Time=1.75 sec
 ARIMA(1,1,1)(2,0,1)[52] intercept   : AIC=

In [26]:
rcc_model.fit(rcc_train['RCC_Outs'])

ARIMA(maxiter=50, method='lbfgs', order=(0, 1, 2), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(2, 0, 0, 52),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [27]:


#HCC model
hcc_model = pm.auto_arima(hcc_new_df.HCC_Outs.iloc[:-1], 
                        m=52, seasonal=False,
                      start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True)



/opt/conda/lib/python3.7/site-packages/pmdarima/arima/_validation.py:62: UserWarning: m (52) set for non-seasonal fit. Setting to 0
  warnings.warn("m (%i) set for non-seasonal fit. Setting to 0" % m)


Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1202.409, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1156.172, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=1200.414, Time=0.02 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=1128.907, Time=0.05 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=1107.339, Time=0.06 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=1101.228, Time=0.11 sec
 ARIMA(5,1,0)(0,0,0)[0] intercept   : AIC=1098.416, Time=0.11 sec
 ARIMA(5,1,1)(0,0,0)[0] intercept   : AIC=1099.186, Time=0.27 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=1097.209, Time=0.15 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=1095.210, Time=0.11 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=1095.599, Time=0.08 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=1097.209, Time=0.18 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=1095.956, Time=0.16 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : 

In [28]:
hcc_model.fit(hcc_train['HCC_Outs'])

ARIMA(maxiter=50, method='lbfgs', order=(3, 1, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=False)

In [29]:


#DTC model
dtc_model = pm.auto_arima(dtc_new_df.DTC_Outs.iloc[:-1], 
                        m=52, seasonal=True,
                      start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True)



Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[52] intercept   : AIC=1062.923, Time=1.15 sec
 ARIMA(0,1,0)(0,0,0)[52] intercept   : AIC=1059.446, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[52] intercept   : AIC=1019.707, Time=0.49 sec
 ARIMA(0,1,1)(0,0,1)[52] intercept   : AIC=973.999, Time=0.63 sec
 ARIMA(0,1,0)(0,0,0)[52]             : AIC=1057.526, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[52] intercept   : AIC=972.003, Time=0.06 sec
 ARIMA(0,1,1)(1,0,0)[52] intercept   : AIC=973.999, Time=0.66 sec
 ARIMA(0,1,1)(1,0,1)[52] intercept   : AIC=976.003, Time=0.91 sec
 ARIMA(1,1,1)(0,0,0)[52] intercept   : AIC=972.462, Time=0.07 sec
 ARIMA(0,1,2)(0,0,0)[52] intercept   : AIC=972.145, Time=0.09 sec
 ARIMA(1,1,0)(0,0,0)[52] intercept   : AIC=1017.721, Time=0.04 sec
 ARIMA(1,1,2)(0,0,0)[52] intercept   : AIC=972.448, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[52]             : AIC=980.095, Time=0.03 sec

Best model:  ARIMA(0,1,1)(0,0,0)[52] intercept
Total fit time: 4.297 seconds


In [30]:
dtc_model.fit(dtc_train['DTC_Outs'])

ARIMA(maxiter=50, method='lbfgs', order=(0, 1, 1), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 52),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [31]:

#EASE model
ease_model = pm.auto_arima(ease_new_df.EASE_Outs.iloc[:-1], 
                        m=52, seasonal=True,
                      start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                           suppress_warnings=True,
                      stepwise=True, trace=True)



Performing stepwise search to minimize aic
 ARIMA(0,0,0)(1,0,1)[52] intercept   : AIC=inf, Time=2.24 sec
 ARIMA(0,0,0)(0,0,0)[52] intercept   : AIC=1245.966, Time=0.02 sec
 ARIMA(1,0,0)(1,0,0)[52] intercept   : AIC=1209.940, Time=1.93 sec
 ARIMA(0,0,1)(0,0,1)[52] intercept   : AIC=1220.775, Time=0.91 sec
 ARIMA(0,0,0)(0,0,0)[52]             : AIC=1649.163, Time=0.02 sec
 ARIMA(1,0,0)(0,0,0)[52] intercept   : AIC=1217.780, Time=0.02 sec
 ARIMA(1,0,0)(2,0,0)[52] intercept   : AIC=1210.914, Time=9.32 sec
 ARIMA(1,0,0)(1,0,1)[52] intercept   : AIC=1210.922, Time=1.17 sec
 ARIMA(1,0,0)(0,0,1)[52] intercept   : AIC=1208.940, Time=0.94 sec
 ARIMA(1,0,0)(0,0,2)[52] intercept   : AIC=1210.914, Time=5.61 sec
 ARIMA(1,0,0)(1,0,2)[52] intercept   : AIC=1212.914, Time=4.78 sec
 ARIMA(0,0,0)(0,0,1)[52] intercept   : AIC=1239.028, Time=0.48 sec
 ARIMA(2,0,0)(0,0,1)[52] intercept   : AIC=1200.622, Time=1.50 sec
 ARIMA(2,0,0)(0,0,0)[52] intercept   : AIC=1207.093, Time=0.08 sec
 ARIMA(2,0,0)(1,0,1)[52]

In [32]:
ease_model.fit(ease_train['EASE_Outs'])


ARIMA(maxiter=50, method='lbfgs', order=(4, 0, 0), out_of_sample_size=0,
      scoring='mse', scoring_args={}, seasonal_order=(1, 0, 0, 52),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)

In [33]:

rcc_forecast =rcc_model.predict(n_periods=52, return_conf_int=True)
hcc_forecast =hcc_model.predict(n_periods=52, return_conf_int=True)
dtc_forecast =dtc_model.predict(n_periods=52, return_conf_int=True)
ease_forecast =ease_model.predict(n_periods=52, return_conf_int=True)



In [34]:
from datetime import date
a = date(date.today().year, 1, 1)

In [35]:

rcc_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')
hcc_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')
dtc_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')
ease_forecast_range=pd.date_range(start=a, periods=52, freq='W-FRI')



In [36]:

rcc_forecast_df = pd.DataFrame(rcc_forecast[0],index= rcc_forecast_range, columns=['Predictions'])
rcc_forecast_df.dropna(inplace=True)
hcc_forecast_df = pd.DataFrame(hcc_forecast[0],index= hcc_forecast_range, columns=['Predictions'])
hcc_forecast_df.dropna(inplace=True)
dtc_forecast_df = pd.DataFrame(dtc_forecast[0],index= dtc_forecast_range, columns=['Predictions'])
dtc_forecast_df.dropna(inplace=True)
ease_forecast_df = pd.DataFrame(ease_forecast[0],index= ease_forecast_range, columns=['Predictions'])
ease_forecast_df.dropna(inplace=True)


In [37]:

rcc_future_forecasts= pd.concat([rcc_new_df['RCC_Outs'].iloc[:-1],rcc_forecast_df],axis=1)
hcc_future_forecasts= pd.concat([hcc_new_df['HCC_Outs'].iloc[:-1],hcc_forecast_df],axis=1)
dtc_future_forecasts= pd.concat([dtc_new_df['DTC_Outs'].iloc[:-1],dtc_forecast_df],axis=1)
ease_future_forecasts= pd.concat([ease_new_df['EASE_Outs'].iloc[:-1],ease_forecast_df],axis=1)


In [38]:
rcc_future_forecasts.head()

,RCC_Outs,Predictions
2021-01-08,978.0,NaN
2021-01-15,898.0,NaN
2021-01-22,759.0,NaN
2021-01-29,909.0,NaN
2021-02-05,968.0,NaN


In [39]:
s_e=rcc_train
s_e=s_e.reset_index()
s= s_e.head(1)
e= s_e.tail(1)

In [40]:

rcc_future_forecasts['Forecast_totals'] = rcc_future_forecasts.fillna(0)['RCC_Outs'] + rcc_future_forecasts.fillna(0)['Predictions']
hcc_future_forecasts['Forecast_totals'] = hcc_future_forecasts.fillna(0)['HCC_Outs'] + hcc_future_forecasts.fillna(0)['Predictions']
dtc_future_forecasts['Forecast_totals'] = dtc_future_forecasts.fillna(0)['DTC_Outs'] + dtc_future_forecasts.fillna(0)['Predictions']
ease_future_forecasts['Forecast_totals'] = ease_future_forecasts.fillna(0)['EASE_Outs'] + ease_future_forecasts.fillna(0)['Predictions']



In [41]:
#RCC

rcc_df_Fut = rcc_future_forecasts.reset_index()
rcc_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
rcc_df_Fut['Year'] = pd.PeriodIndex(rcc_df_Fut.Date, freq='Y')
rcc_df_Fut['Quarter'] = rcc_df_Fut.Date.dt.quarter
weekend= rcc_master_dataframe_cabo.tail(2)
rcc_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
rcc_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
rcc_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
rcc_df_Fut['Sales_Type'] = 'COMMERCIAL'
rcc_df_Fut['Model_Type'] = 'RCC-ARIMA'



In [42]:
#HCC

hcc_df_Fut = hcc_future_forecasts.reset_index()
hcc_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
hcc_df_Fut['Year'] = pd.PeriodIndex(hcc_df_Fut.Date, freq='Y')
hcc_df_Fut['Quarter'] = hcc_df_Fut.Date.dt.quarter
weekend= hcc_master_dataframe_cabo.tail(2)
hcc_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
hcc_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
hcc_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
hcc_df_Fut['Sales_Type'] = 'COMMERCIAL'
hcc_df_Fut['Model_Type'] = 'HCC-ARIMA'



In [43]:
#DTC

dtc_df_Fut = dtc_future_forecasts.reset_index()
dtc_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
dtc_df_Fut['Year'] = pd.PeriodIndex(dtc_df_Fut.Date, freq='Y')
dtc_df_Fut['Quarter'] = dtc_df_Fut.Date.dt.quarter
weekend= dtc_master_dataframe_cabo.tail(2)
dtc_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
dtc_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
dtc_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
dtc_df_Fut['Sales_Type'] = 'COMMERCIAL'
dtc_df_Fut['Model_Type'] = 'DTC-ARIMA'



In [44]:
#EASE

ease_df_Fut = ease_future_forecasts.reset_index()
ease_df_Fut.rename(columns = {'index':'Date'}, inplace = True)
ease_df_Fut['Year'] = pd.PeriodIndex(ease_df_Fut.Date, freq='Y')
ease_df_Fut['Quarter'] = ease_df_Fut.Date.dt.quarter
weekend= ease_master_dataframe_cabo.tail(2)
ease_df_Fut['Week_Ending_date'] = weekend['week_end_date'].iloc[0]
ease_df_Fut['Training_Start_date'] = s['week_end_date'].iloc[0]
ease_df_Fut['Training_End_date'] = e['week_end_date'].iloc[0]
ease_df_Fut['Sales_Type'] = 'EASE'
ease_df_Fut['Model_Type'] = 'EASE-ARIMA'



In [45]:

rcc_hcc_one = rcc_df_Fut.append(hcc_df_Fut)
rcc_hcc_dtc = rcc_hcc_one.append(dtc_df_Fut)
all_one = rcc_hcc_dtc.append(ease_df_Fut)



In [46]:
final_df= all_one[['Date','Year','Quarter','Week_Ending_date','Sales_Type','Model_Type','Training_Start_date','Training_End_date','RCC_Outs','HCC_Outs','DTC_Outs','EASE_Outs','Predictions','Forecast_totals']]

In [47]:
final_df.reset_index(drop=True, inplace=True)

In [48]:
s_e=final_df
s_e=s_e.reset_index()
e= s_e.Week_Ending_date.iloc[0]

In [49]:
file_name_1 = "ARIMA_Forecast.csv"
final_df.to_csv(file_name_1,index=False)

In [50]:
a = 'ARIMA_Forecasts.csv'
b = str(e)

In [51]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file(file_name_1, 'forecastingpoc', 'inputfiles/'+b+' - '+a)